In [1]:
import os
import datetime
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

import numpy as np
import pandas as pd

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

In [4]:
base_model = MobileNet(weights='imagenet', include_top=False) 

Instructions for updating:
Colocations handled automatically by placer.


/home/marcolini/anaconda3/envs/deep_myq/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [5]:
DATASET_DIR = '/data3/marcolini/myq_dataset/Selfie-dataset/labelled/'

In [6]:
images = os.listdir(DATASET_DIR)
images = pd.DataFrame(images)
images = images.rename(index=int, columns={0: "filename"})
images = images.sample(frac=1).reset_index(drop=True)

In [7]:
images.head()

,filename
0,0_0_4_10249109_610036962415333_2136693797_a.jpg
1,1_1_2_1171148_524036517705698_54959346_a.jpg
2,1_1_3_1530850_505290662908732_941927508_a.jpg
3,0_1_3_10013202_748090675236234_247501991_a.jpg
4,2_1_3_10005480_320200204794412_2034690271_a.jpg


In [8]:
images['negative'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '0')))
images['neutral'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '1')))
images['positive'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '2')))

images['male'] = images['filename'].apply(lambda x: (int(x.split('_')[1] == '0')))
images['female'] = images['filename'].apply(lambda x: (int(x.split('_')[1] == '1')))

images['baby'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '0')))
images['child'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '1')))
images['teenager'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '2')))
images['youth'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '3')))
images['middle_age'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '4')))
images['senior'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '5')))

In [9]:
images.sum()

filename      0_0_4_10249109_610036962415333_2136693797_a.jp...
negative                                                   4244
neutral                                                   16453
positive                                                  12170
male                                                       8770
female                                                    24099
baby                                                         96
child                                                       690
teenager                                                   4945
youth                                                     26143
middle_age                                                  982
senior                                                       13
dtype: object

In [10]:
images.columns[1:]

Index(['negative', 'neutral', 'positive', 'male', 'female', 'baby', 'child',
       'teenager', 'youth', 'middle_age', 'senior'],
      dtype='object')

In [11]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [12]:
columns = images.columns[1:].tolist()

In [13]:
n_train = len(images)//4 *3

In [15]:
train_generator = train_datagen.flow_from_dataframe(dataframe=images[:n_train], 
                                                    directory=DATASET_DIR, 
                                                    x_col='filename', 
                                                    y_col=columns,
                                                    class_mode='other',
                                                    target_size=(224,224),
                                                    color_mode='rgb')

test_generator = test_datagen.flow_from_dataframe(dataframe=images[n_train:], 
                                                    directory=DATASET_DIR, 
                                                    x_col='filename', 
                                                    y_col=columns,
                                                    class_mode='other',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    shuffle=False)

Found 24651 images.
Found 8218 images.


In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x) 

output_layers = []
for i in range(11):
    output_layers.append(Dense(1, activation = 'sigmoid')(x))

model = Model(inputs=base_model.input, outputs=output_layers)

In [17]:
for layer in model.layers[:87]:
    layer.trainable=False
for layer in model.layers[87:]:
    layer.trainable=True

In [ ]:
model.summary()

In [18]:
opt = tf.train.AdamOptimizer()

In [19]:
model.compile(optimizer=opt, loss=['binary_crossentropy']*11, metrics=['accuracy'])

In [20]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x, [batch_y[:,i] for i in range(11)])

In [ ]:
next(generator_wrapper(train_generator))[0]

In [21]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size

In [23]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=False)

In [ ]:
model.fit_generator(generator=generator_wrapper(train_generator),
                    steps_per_epoch=step_size_train,
                    validation_data=(generator_wrapper(test_generator)),
                    validation_steps=step_size_test,
                    callbacks=[tensorboard_callback],
                    verbose=0,
                    epochs=20)

Instructions for updating:
Use tf.cast instead.
  8/256 [..............................] - ETA: 16:17 - loss: 4.4535 - dense_3_loss: 0.5485 - dense_4_loss: 0.7613 - dense_5_loss: 0.8235 - dense_6_loss: 0.4825 - dense_7_loss: 0.4607 - dense_8_loss: 0.0371 - dense_9_loss: 0.1471 - dense_10_loss: 0.4587 - dense_11_loss: 0.5588 - dense_12_loss: 0.1753 - dense_13_loss: 2.2355e-05 - dense_3_acc: 0.8672 - dense_4_acc: 0.6016 - dense_5_acc: 0.5234 - dense_6_acc: 0.8125 - dense_7_acc: 0.8125 - dense_8_acc: 0.9922 - dense_9_acc: 0.9766 - dense_10_acc: 0.8789 - dense_11_acc: 0.8164 - dense_12_acc: 0.9688 - dense_13_acc: 1.0000

In [ ]:
model.save_weights('weights/mobilenet20_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), save_format='tf')
save_model(model, 'weights_h5/mobilenet20_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".h5")

In [ ]:
for layer in model.layers[:87]:
    layer.trainable=False
for layer in model.layers[87:]:
    layer.trainable=True

In [ ]:
img = image.load_img('opencv_image.JPG', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

In [ ]:
np.argmax(preds[:3]) # emotions

In [ ]:
np.argmax(preds[3:5]) # gender

In [ ]:
np.argmax(preds[5:]) # age class